In [1]:
from matplotlib.pyplot import *
import numpy as np
import pandas as pd
import os
import importlib

from ATARI.sammy_interface import sammy_classes, sammy_functions

from ATARI.ModelData.particle_pair import Particle_Pair
# from ATARI.ModelData.experimental_model import Experimental_Model
from ATARI.syndat.control import Syndat_Control

from copy import copy

import ATARI.utils.plotting as myplot


In [2]:
sammypath = ''
assert(sammypath != '')

In [3]:
%matplotlib widget

# Overview

Syndat models for each measurement were developed, investigated, and saved in the associated example notebooks.
Here, we are going to load each Syndat model, add them to the Syndat Control module, and draw samples from them.
A few things to note, 
1) options should likely be re-defined as we want to ensure resonance and parameter sampling is turned on
2) the energy ranges can be overwritten s.t. you can draw samples for a smaller window.
3) The background function is the same for all 3 the transmission measurements, this can be implemented with the model correlations input


In [4]:
from ATARI.utils import atario

syndat_trans1mm = atario.load_syndat(os.path.join(os.getcwd(), "results", "SyndatModel_1mmT.pkl"))
syndat_trans3mm = atario.load_syndat(os.path.join(os.getcwd(), "results", "SyndatModel_3mmT.pkl"))
syndat_trans6mm = atario.load_syndat(os.path.join(os.getcwd(), "results", "SyndatModel_6mmT.pkl"))

syndat_cap1mm = atario.load_syndat(os.path.join(os.getcwd(), "results", "SyndatModel_1mmY.pkl"))
syndat_cap2mm = atario.load_syndat(os.path.join(os.getcwd(), "results", "SyndatModel_2mmY.pkl"))

In [5]:
# energy_range_all = [197.5, 235]
energy_range_all = [200, 3e3]


Ta_pair = Particle_Pair(isotope="Ta181", formalism="XCT", energy_range=energy_range_all, 
                        ac=0.81271, M=180.948030, m=1, I=3.5, i=0.5, l_max=1)      

Ta_pair.add_spin_group(Jpi='3.0', J_ID=1,
                       D=9.0030,
                       gn2_avg=452.56615, gn2_dof=1,
                       gg2_avg=32.0, gg2_dof=100)

Ta_pair.add_spin_group(Jpi='4.0', J_ID=2,
                       D=8.3031,
                       gn2_avg=332.24347, gn2_dof=1,
                       gg2_avg=32.0, gg2_dof=100)

In [6]:
## get new syndat models in a smaller window

syndat_models = [syndat_trans1mm, syndat_trans3mm, syndat_trans6mm, syndat_cap1mm, syndat_cap2mm]
syndat_models_new = []
for each in syndat_models:
    syndat_models_new.append(each.truncate_energy_range(energy_range_all, return_copy=True))


In [7]:

rto = sammy_classes.SammyRunTimeOptions(sammypath,
                             **{"Print"   :   True,
                              "bayes"   :   False,
                              "keep_runDIR"     : False,
                              "sammy_runDIR": "sammy_runDIR_gen"
                              })

## define model correlation for a_b parameters
trans_a_b = syndat_trans1mm.generative_measurement_model.model_parameters.a_b
model_correlations = [
                    # {'models': [1,1,1,0,0],
                    # 'a_b'   : trans_a_b }
                        ]

syndat = Syndat_Control(Ta_pair,
                        syndat_models = syndat_models_new,
                        model_correlations=model_correlations,
                        sampleRES=True,
                        save_covariance=True,
                        save_raw_data=False)

In [8]:
syndat.sample(rto, num_samples=10)

In [17]:
sample1 = syndat.get_sample(-5)
datasets = [val.pw_reduced for key, val in sample1.items()]
experiments = [val.generative_experimental_model for val in syndat_models]

# importlib.reload(myplot)
# fig = myplot.plot_reduced_data_TY(datasets=datasets,
#                             experiments=experiments,
#                             xlim=energy_range_all,
#                             plot_datasets_true=True
#                             )

In [ ]:

colors = ["C1", "C2", "C3", "C4", "C5", "C6", "C7"]
alphas = [1.0, 0.75, 0.5, 1.0, 0.5]
fig, axes = subplots(2,1, figsize=(8,6), sharex=True)

for i, exp in enumerate(experiments):

    # if i in [1,4,3]:
    #     continue
    
    if exp.reaction == "transmission":
        model_key = "theo_trans"
        iax = 0
        loc = 'lower right'
        # ylim = (-0.5,1.3)
    elif exp.reaction == "capture":
        model_key = "theo_xs"
        iax = 1
        loc='upper right'
        # ylim = (-0.2,1.8)
    else:
        raise ValueError()

    axes[iax].errorbar(datasets[i].E, datasets[i].exp, yerr=datasets[i].exp_unc, zorder=2,
                                            fmt='.', color='black', alpha=alphas[i], linewidth=0.5, markersize=1, capsize=1, label=exp.title)

lines = axes[1].get_lines()

axes[0].set_ylim(-0.15, 1.1)
axes[1].set_ylim(-0.1, 1.0)

axes[0].legend(loc="lower right")
axes[1].legend(loc="upper right")

for ax in axes:
    ax.set_xlim((200,3000))
    
    # ax.set_facecolor('#0f0f0f10')  
    # ax.grid(True, linestyle='-', color='white', alpha=1.0, zorder=0)  # Add grid with style
    # ax.tick_params(axis='both', which='both', length=0, width=0)

axes[0].set_ylabel("Transmission")#(1mm)")
axes[1].set_ylabel("Capture Yield")# (1mm)")
# fig.subplots_adjust(wspace=0, hspace=0.05)
fig.supxlabel('Energy (eV)')
fig.suptitle("Ta-181 Measurement Data and Fitted Models")
fig.tight_layout()

## testing the noise distribution of your syndat model
You can use the noise distribution test to verify the validity of linearity and normality assumptions in the propagation of measurement uncertainty.
If these assumptions are met, then the normalized residual should fall on a standard normal and the chi2 statistic should fall on a chi2 distribution with degrees of freedom equal to the number of data points.
This function performs this test for 10 randomly sampled "true experimental values" between 0 and 1 (the possible range for transmission or capture yield). 
For this reason, it is important to pass a copy of your syndat models to the function s.t. the settings to get changed.
For these Syndat_Models, the assumptions are not fully met, particularly for the transmission datasets.
From additional investigations, this was found to be due primarily to the power background function.
In the limit of very small uncertainty on measurement parameters, these assumptions will improve (see ATARI.tests.benchmarks.test_syndat_distributions).

In [ ]:
from ATARI.syndat.tests import noise_distribution_test, noise_distribution_test2
from copy import deepcopy

mean = []
norm_test = []
kstest = []
for each in syndat_models_new:
    mean_of_residual, norm_test_on_residual, kstest_on_chi2 = noise_distribution_test(deepcopy(each), print_out=True, ipert=250, energy_range = [150,2500])
    mean.append(mean_of_residual)
    norm_test.append(norm_test_on_residual)
    kstest.append(kstest_on_chi2)

# Saving the Syndat Control module and all samples
Similar to the syndat_models, you can use the atario function to save the syndat_control object as a pickle. 
This is commented out because an existing pkl file is loaded

In [26]:

# atario.save_syndat_control(syndat, os.path.join(os.getcwd(), "results", "SyndatModel_All_200_235.pkl"))